# Automobile Sales Dashboard — Interactive Dash App

**Short description:**  
This notebook builds a Dash interactive dashboard to explore historical automobile sales, showing both yearly statistics and recession-period analyses with multiple interactive visualizations.

**Objectives**
- Load and inspect historical automobile sales data.
- Build an interactive Dash dashboard with dropdown controls for report selection and year selection.
- Visualize yearly trends, monthly breakdowns, vehicle-type analyses, advertising expenditure shares, and the effect of unemployment on sales.
- Practice implementing Dash callbacks to enable dynamic interactions between controls and visualizations.

**Notice about documentation:**  
The original notebook submission (course assignment) was kept intact. I have **only modified documentation (comments, headings, markdown)** and made **minimal, necessary corrections** to ensure the notebook runs without errors. All rights related to the lab/workshop design and original exercise belong exclusively to **IBM Corporation**. This notebook includes additional documentation for clarity, but the intellectual property of the original exercise is retained by IBM.

---

## Table of contents

1. Dependencies & execution instructions  
2. Data loading & inspection  
3. App layout — UI components (dropdowns, year selector, output container)  
4. Callbacks — enabling interactivity  
5. Recession Period Statistics (graphs)  
6. Yearly Statistics (graphs)  
7. Running the app & notes on deployment

## 1) Dependencies & execution instructions

This section installs and imports required Python packages.  

**Recommended local execution steps:**

1. Create and activate a Python virtual environment:
   - `python -m venv venv`
   - `source venv/bin/activate` (macOS / Linux) or `venv\Scripts\activate` (Windows)
2. Install dependencies:
   - `pip install -r requirements.txt`
3. Run the dashboard:
   - `python Automobile_Statistics_Dashboard_App.py`

**Local execution notes**

If running from a notebook, ensure that `%run` or appropriate cell execution is used. Dash apps are usually run as scripts.

In [ ]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px


## 2) Data loading & inspection

This section loads the historical automobile sales CSV into a pandas DataFrame from the provided URL and performs minimal inspection. The dataset is used by the Dash app for all visualizations and computations.

In [ ]:
# Load the data using pandas
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

# Initialize the Dash app
app = dash.Dash(__name__)

## 3) App layout — UI components

This section defines the Dash app layout: the title, dropdown to select report type, year selector, and the output container where graphs will be rendered. Styling for components is included inline.

In [ ]:
# Create the dropdown menu options
dropdown_options = [
    {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
    {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
]
# List of years 
year_list = [i for i in range(1980, 2024, 1)]

#---------------------------------------------------------------------------------------

# Create the layout of the app

app.layout = html.Div([
    # Title of the dashboard
    html.H1("Automobile Sales Statistics Dashboard",style={'textAlign': 'center', 'color': '#503D36','font-size': 24}),#May include style for title
    html.Div([# Dropdown menus
        html.Label("Select Statistics:"),
        dcc.Dropdown(
            id='dropdown-statistics',
            options=dropdown_options,
            value='Select Statistics',
            placeholder='Select a report type',
            style={'width':'80%','padding':'3px','font-size':'20px','text-align-last':'center'}
        )
    ]),
    html.Div(dcc.Dropdown(
            id='select-year',
            options=[{'label': i, 'value': i} for i in year_list],
            placeholder='Select a year',
            style={'width':'80%','padding':'3px','font-size':'20px','text-align-last':'center'},
            value='Select-year'
        )),
    html.Div([# Division for output display
    html.Div(id='output-container', className='chart-grid', style={'display':'flex'}),])
])

## 4) Callbacks — enabling interactivity

This section contains Dash callbacks:
- One callback enables/disables the year selector based on the selected report type.
- Another callback renders appropriate graphs (children of the output container) depending on the report type and selected year.

In [ ]:
# Define the callback function to update the input container based on the selected statistics
@app.callback(
    Output(component_id='select-year', component_property='disabled'),
    Input(component_id='dropdown-statistics',component_property='value'))

def update_input_container(selected_statistics):
    if selected_statistics =='Yearly Statistics': 
        return False
    else: 
        return True

#Callback for plotting
# Define the callback function to update the input container based on the selected statistics
@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='dropdown-statistics', component_property='value'), 
    Input(component_id='select-year', component_property='value')])

## 5) Recession Period Statistics and Yearly Statistics

When "Recession Period Statistics" is selected, the app filters rows where `Recession == 1` and displays:
- Average automobile sales over recession years (line chart),
- Average sales by vehicle type (bar chart),
- Advertising expenditure share by vehicle type (pie chart),
- Effect of unemployment rate on sales by vehicle type (segmented bar chart).

When "Yearly Statistics" is selected and a `Year` is chosen, the app shows:
- Average automobile sales over time (line chart),
- Monthly automobile sales for the selected year (line chart),
- Average vehicles sold by vehicle type for that year (bar chart),
- Total advertising expenditure share by vehicle type for that year (pie chart).

In [ ]:
def update_output_container(selected_statistics, input_year):
    if selected_statistics == 'Recession Period Statistics':
        # Filter the data for recession periods
        recession_data = data[data['Recession'] == 1]

        # Plot 1 Automobile sales fluctuate over Recession Period (year wise)
        yearly_rec=recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1 = dcc.Graph(
            figure=px.line(yearly_rec, 
                x='Year',
                y='Automobile_Sales',
                title="Average Automobile Sales fluctuation over Recession Period"))

        # Plot 2 Calculate the average number of vehicles sold by vehicle type       
        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()                           
        R_chart2  = dcc.Graph(
            figure=px.bar(average_sales, 
                x='Vehicle_Type',
                y='Automobile_Sales',
                title="Vehicle-Wise Average Automobile Sales during Recession Period"))
        
        # Plot 3 Pie chart for total expenditure share by vehicle type during recessions
        exp_rec= recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(
                    figure=px.pie(exp_rec,
                    values='Advertising_Expenditure',
                 names='Vehicle_Type',
                 title="Total Expenditure Share by Vehicle Type during Recessions"
                )
        )

        # Plot 4 bar chart for the effect of unemployment rate on vehicle type and sales
        unemp_data= recession_data.groupby(['unemployment_rate', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
        R_chart4 = dcc.Graph(
            figure=px.bar(
                unemp_data,
                x='unemployment_rate',
                y='Automobile_Sales',
                color='Vehicle_Type',
                labels={'unemployment_rate': 'Unemployment Rate', 'Automobile_Sales': 'Average Automobile Sales'},
                title="Effect of Unemployment Rate on Sales of various Vehicle Types")
        )


        return [
            html.Div(className='chart-item', children=[html.Div(children=R_chart1),html.Div(children=R_chart2)],style={'display': 'flex'}),
            html.Div(className='chart-item', children=[html.Div(children=R_chart3),html.Div(children=R_chart4)],style={'display': 'flex'})
            ]

    # Yearly Statistic Report Plots  
    #                            
    elif (input_year and selected_statistics=='Yearly Statistics') :
        yearly_data = data[data['Year'] == input_year]
                              
        # Plot 1 Yearly Automobile sales using line chart for the whole period.
        yas= data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(
            figure=px.line(yas, 
                x='Year',
                y='Automobile_Sales',
                title="Average Automobile Sales over Time"))
            
        # Plot 2 Total Monthly Automobile sales using line chart.
        Y_chart2 = dcc.Graph(
            figure=px.line(yearly_data, 
                x='Month',
                y='Automobile_Sales',
                title="Monthly Automobile sales in the year {}".format(input_year)))

        # Plot bar chart for average number of vehicles sold during the given year
        avr_vdata=yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        Y_chart3 = dcc.Graph(
            figure=px.bar(avr_vdata, 
                x='Vehicle_Type',
                y='Automobile_Sales',
                title='Average Vehicles Sold by Vehicle Type in the year {}'.format(input_year)))

        # Total Advertisement Expenditure for each vehicle using pie chart
        exp_data=yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        Y_chart4 = dcc.Graph(figure=px.pie(exp_data,
                    values='Advertising_Expenditure',
                 names='Vehicle_Type',
                 title="Total Expenditure Share by Vehicle Type"
                )
        )

        return [
                html.Div(className='chart-item', children=[html.Div(children=Y_chart1),html.Div(children=Y_chart2)],style={'display': 'flex'}),
                html.Div(className='chart-item', children=[html.Div(children=Y_chart3),html.Div(children=Y_chart4)],style={'display': 'flex'})
                ]
        
    else:
        return None

## 6) Running the app & deployment notes

- To run locally, execute the script (or use the `if __name__ == '__main__': app.run()` cell).  
- Ensure the dataset URL is accessible from the deployment environment or include a local copy of the CSV in the project.

In [ ]:
# Run the Dash app
if __name__ == '__main__':
    app.run()